<a href="https://colab.research.google.com/github/redromnon/gif2gif-sd/blob/main/gif2gif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install moviepy pillow diffusers transformers xformers git+https://github.com/huggingface/accelerate.git controlnet_aux

In [ ]:
# @title Setup and Model Selection
from PIL import Image
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, UniPCMultistepScheduler, StableDiffusionImg2ImgPipeline, EulerAncestralDiscreteScheduler
import torch, mediapy
import numpy as np
from diffusers.utils import load_image
from diffusers.models import AutoencoderKL
import torch

from controlnet_aux import CannyDetector
canny = CannyDetector()

#SD
model = "" #@param {type:"string"}

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)


#Using Img2Img ControlNet pipeline
pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
    model,
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16,
    vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
)

pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.enable_vae_slicing()
pipe.enable_xformers_memory_efficient_attention()
pipe.to("cuda")

del controlnet

In [ ]:
# @title Select GIF
uri = "" #@param {type:"string"}
#@markdown Paste the link to the GIF file <br/>
#@markdown **Make sure the GIF image is in 1:1 or square ratio**
#Read GIF

from PIL import Image
from PIL import GifImagePlugin
from diffusers.utils import load_image
import mediapy, requests
from moviepy.editor import VideoFileClip


with open('sample.gif', 'wb') as f:
  f.write(requests.get(uri).content)

imageObject = Image.open("./sample.gif")

print(f"Is GIF? {imageObject.is_animated}")

print(f"Total frames in GIF: {imageObject.n_frames}")


In [ ]:
# @title Adjust Canny threshold
frame = 0 #@param {type:"integer"}
low_threshold = 50 #@param {type:"number"}
high_threshold = 120 #@param {type:"number"}
#@markdown Apply and test canny edge detection on a particular frame


#CHECK CONTROLNET CONDITION
imageObject.seek(frame)

#Convert GIF file type to Image file type
img = Image.new('RGB', imageObject.size)
img.paste(imageObject)

#Canny
control_image = canny(img, low_threshold=low_threshold, high_threshold=high_threshold)
control_image

**Prompts**

In [ ]:
prompt = """
<insert you prompt here>
"""

n_prompt = """
<insert things to avoid>
"""

In [ ]:
# @title Generate
#@markdown The range of frames to consider from the GIF:
min_frame = 0 #@param {type:"number"}
max_frame = 10 #@param {type:"number"}
#@markdown Pipeline parameters:
num_inference_steps = 20 #@param {type:"slider", min:0, max:50, step:1}
guidance_scale = 7.5 #@param {type:"slider", min:0, max:20, step:0.5}
strength = 0.3 #@param {type:"slider", min:0, max:1, step:0.1}
controlnet_conditioning_scale = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}


# Select individual frames from the loaded animated GIF file
frames=[]

for frame in range(min_frame, max_frame):

    #Save frames
    imageObject.seek(frame)

    #Convert GIF file type to Image file type
    img = Image.new('RGB', imageObject.size)
    img.paste(imageObject)

    frames.append(img.resize((512,512)))


###################################################################
#RUN
import random

#Set random seed
seed= random.randint(0, 2147483647)
#seed = 123


output_frames = []


#RUN
for frame in frames:

  #Convert frame to lineart
  control_image = canny(frame, low_threshold, high_threshold)


  #Run pipeline
  output = pipe(
      prompt,
      image=frame,
      control_image=control_image,
      negative_prompt=n_prompt,
      num_inference_steps=num_inference_steps,
      generator=torch.manual_seed(seed),
      guidance_scale=guidance_scale,
      strength=strength,
      controlnet_conditioning_scale=controlnet_conditioning_scale
  ).images[0]

  #Append
  output_frames.append(output)

In [ ]:
# @title Convert to GIF
fps = 3 #@param {type:"slider", min:0, max:30, step:1}
#@markdown To **view** the newly generated GIF, click the files icon on the left-hand side and select **"clip.gif"** <br/>
#@markdown You also have an option **download** "clip.gif" to your computer

#Convert to GIF
import moviepy.editor as mpy

# Convert each PIL.Image.Image object to a NumPy array
frames = [np.array(img) for img in output_frames]

clip = mpy.ImageSequenceClip(frames, fps=fps)
clip.write_gif('{}.gif'.format("clip"), fps=fps)